In [1]:
import os
os.chdir("..")

In [2]:
import hydra
from omegaconf import DictConfig

from shell.fleet.helper import get_fleet, get_agent_cls
import torch
import time
import datetime
import logging
from shell.utils.experiment_utils import setup_experiment, process_dataset_cfg, eval_net
from shell.datasets.datasets import get_dataset
from pprint import pprint
import ray
import os
from hydra import compose, initialize
from omegaconf import OmegaConf
from shell.utils.utils import seed_everything
from shell.fleet.model_sharing_utils import is_in, diff_models
logging.basicConfig(level=logging.INFO)

In [3]:
initialize(config_path="conf", job_name="tmp_job")
cfg = compose(config_name="grad_mono")
seed_everything(cfg.seed)

/tmp/ipykernel_4165783/178382995.py:1: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path="conf", job_name="tmp_job")
/home/vlongle/miniconda3/envs/shell/lib/python3.10/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'grad_mono': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [4]:
# override
cfg.parallel = True

In [5]:
cfg.train.num_epochs = 50
cfg.train.component_update_freq = 50

In [6]:
AgentCls = get_agent_cls(cfg.sharing_strategy, cfg.algo, cfg.parallel)

graph, datasets, NetCls, LearnerCls, net_cfg, agent_cfg, train_cfg = setup_experiment(
        cfg)
FleetCls = get_fleet(cfg.sharing_strategy, cfg.parallel)

INFO:root:Class sequence: [4 7 0 8 4 6 4 3 8 0 1 0 5 9 6 4 9 0 3 0]
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)


{'train': {'component_update_freq': 50, 'num_epochs': 50, 'save_freq': 1}, 'dataset': {'dataset_name': 'mnist', 'num_tasks': 10, 'num_classes_per_task': 2, 'with_replacement': True, 'num_trains_per_class': 64, 'num_vals_per_class': 50, 'remap_labels': True}, 'net': {'name': 'mlp', 'depth': 2, 'layer_size': 64, 'dropout': 0.0}, 'sharing_strategy': {'name': 'gradient', 'num_coms_per_round': 5, 'retrain': {'num_epochs': 10}}, 'seed': 0, 'algo': 'monolithic', 'job_name': 'fun', 'num_agents': 4, 'root_save_dir': 'results', 'parallel': True, 'num_init_tasks': 2, 'agent': {'save_dir': '${root_save_dir}/${job_name}/${dataset.dataset_name}/${algo}/seed_${seed}', 'batch_size': 64, 'memory_size': 64, 'improvement_threshold': 0.05, 'use_contrastive': True}}


INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:Class sequence: [2 1 3 8 6 0 3 0 5 0 3 6 2 7 6 7 6 1 0 5]
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:Class sequence: [0 9 7 2 1 8 6 1 6 4 5 7 8 0 2 3 0 3 9 6]
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:Class sequence: [0 5 3 4 4 5 9 3 0 6 9 1 3 1 7 9 5 2 3 2]
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(12

i_size 28
num_classes 2
net_cfg {'name': 'mlp', 'depth': 2, 'layer_size': 64, 'dropout': 0.0, 'i_size': 28, 'num_classes': 2, 'num_tasks': 10, 'num_init_tasks': 2}
<class 'shell.learners.er_nocomponents.NoComponentsER'>


In [7]:
fake_dataset = get_dataset(**process_dataset_cfg(cfg))
fleet = FleetCls(graph, cfg.seed, datasets, cfg.sharing_strategy, AgentCls, NetCls=NetCls,
                     LearnerCls=LearnerCls, net_kwargs=net_cfg, agent_kwargs=agent_cfg,
                     train_kwargs=train_cfg, fake_dataset=fake_dataset)

INFO:root:Class sequence: [3 9 2 4 9 2 5 4 8 9 9 4 1 6 9 6 8 0 5 1]
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
2023-03-28 18:05:05,008	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
(ParallelModelSyncAgent pid=4173963) INFO:root:Agent: node_id: 69420, seed: 69420000
(ParallelModelSyncAgent pid=4173963) INFO:root:epochs: 0, training task: 0
(ParallelModelSyncAgent pid=4173963) INFO:root:	task: 0	loss: 6.234	acc: 0.500
(ParallelModelSyncAgent pid=4173963) INFO:root:	task: avg	loss: 6.234	acc: 0.500
(ParallelModelSyncAgent pid=4173963) INFO:root:epochs: 0, training task: 0
(ParallelModelSyncAgent pid=4173963) INFO:root:	task: 0	loss: 6.232	acc: 0.500
(ParallelModelSyncAgent pid=4173963) INFO:root:	task: avg	loss: 6.232	acc

In [ ]:
[dataset.class_sequence for dataset in datasets]

In [ ]:
fake_dataset.class_sequence

In [ ]:
# diff_models(fleet.agents[0].get_net().state_dict(), fleet.agents[1].get_net().state_dict(),
#             keys=["random_linear_projection", "components"])

print("After initialization:")
diff_models(ray.get(fleet.agents[0].get_net.remote()).state_dict(), 
            ray.get(fleet.agents[1].get_net.remote()).state_dict(),
            keys=["random_linear_projection", "components"])

In [ ]:
print("Accuracy just after initialization:")
# evaluate the performance again
for agent in fleet.agents:
    print(ray.get(agent.eval.remote(0)))

In [ ]:
# local train
fleet.train(0)

In [ ]:
print("After local training:")

diff_models(ray.get(fleet.agents[0].get_net.remote()).state_dict(), 
            ray.get(fleet.agents[1].get_net.remote()).state_dict(),
            keys=["random_linear_projection", "components"])

In [ ]:
print("Accuracy after local train:")
# evaluate the performance again
for agent in fleet.agents:
    print(ray.get(agent.eval.remote(0)))

In [ ]:
fleet.num_coms_per_round = 10
fleet.communicate(0)

In [ ]:
diff_models(ray.get(fleet.agents[0].get_net.remote()).state_dict(), 
            ray.get(fleet.agents[1].get_net.remote()).state_dict(),
            keys=["random_linear_projection", "components"])

In [ ]:
print("Acc after sync")
# evaluate the performance again
for agent in fleet.agents:
    print(ray.get(agent.eval.remote(0)))